In [1]:
import numpy as np
import tensorflow as tf
import import_ipynb
from params import model_params
p = model_params('config.ini')

def freq2erb (freq_hz):
    return 9.265 * np.log(1 + freq_hz / (24.7 * 9.265))

def erb2freq (n_erb):
    return 24.7 * 9.265 * (np.exp(n_erb / 9.265) -1)

def ERBBand(N, high_lim, NFFT):
    freqRangePerBin = high_lim / (NFFT/2)
    erb_low = freq2erb(0)
    erb_high = freq2erb(high_lim)
    erb_lims = np.linspace(erb_low, erb_high, N)
    
    cutoffs = [erb2freq(erb_lims[i]) for i in range (N)]
    
    nfftborder = [np.round((cutoffs[i]+(freqRangePerBin/2))/freqRangePerBin) for i in range (N)]

    for i in range (N-2):
        if (nfftborder[i+2] - nfftborder[i]) < 2:
            nfftborder[i+2] += (2-(nfftborder[i+2] - nfftborder[i]))
    
    return nfftborder


def ERB_pro_matrix(nfftborder, NFFT, mode):
    N                 = len(nfftborder)
    ERB_Matrix        = np.zeros((int(NFFT / 2 +1) , N))
    
    for i in range(N - 1):
        band_size        = nfftborder[i+1]-nfftborder[i]
        
        for j in range(int(band_size)):
            ERB_Matrix[int(nfftborder[i])+j, i]  = 1 - j / band_size
            ERB_Matrix[int(nfftborder[i])+j,i+1] = j / band_size
    if mode==0:
        ERB_Matrix[:, 0]   *=2
        ERB_Matrix[:, N-1] *=2

    return ERB_Matrix

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


importing Jupyter notebook from params.ipynb


In [2]:
X = np.ones(513)
ERBB = ERBBand(N=32, high_lim=24000, NFFT=1024)
ERBB

[0.0,
 1.0,
 2.0,
 3.0,
 5.0,
 6.0,
 8.0,
 10.0,
 12.0,
 15.0,
 18.0,
 21.0,
 25.0,
 30.0,
 36.0,
 42.0,
 50.0,
 59.0,
 69.0,
 81.0,
 94.0,
 110.0,
 129.0,
 151.0,
 176.0,
 205.0,
 239.0,
 279.0,
 325.0,
 378.0,
 440.0,
 513.0]

In [3]:
import matplotlib.pyplot as plt

In [4]:
# ERB_Matrix = ERB_pro_matrix(ERBB, NFFT=1024, mode=0) #  ERB convert matrix
# iERB_Matrix = ERB_pro_matrix(ERBB, NFFT=1024, mode=1) #  ERB convert matrix
# plt.plot(ERB_Matrix)
# plt.show()
# plt.plot(iERB_Matrix)
# plt.show()

In [5]:
ERB_Matrix = ERB_pro_matrix(ERBB, NFFT=1024, mode=0) #  ERB convert matrix
Shane_B = np.sqrt(X @ ERB_Matrix)
Shane_B

array([1.41421356, 1.        , 1.        , 1.22474487, 1.22474487,
       1.22474487, 1.41421356, 1.41421356, 1.58113883, 1.73205081,
       1.73205081, 1.87082869, 2.12132034, 2.34520788, 2.44948974,
       2.64575131, 2.91547595, 3.082207  , 3.31662479, 3.53553391,
       3.80788655, 4.18330013, 4.52769257, 4.84767986, 5.19615242,
       5.61248608, 6.08276253, 6.55743852, 7.03562364, 7.58287544,
       8.21583836, 8.48528137])

In [7]:
X = np.ones(64)
iERB_Matrix = ERB_pro_matrix(ERBB, NFFT=1024, mode=1) # inverse ERB convert matrix
# X @ iERB_Matrix.T

In [8]:
iERB_Matrix.T

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.04109589, 0.02739726,
        0.01369863],
       [0.        , 0.        , 0.        , ..., 0.95890411, 0.97260274,
        0.98630137]])

In [ ]:
# iERB_Matrix = ERB_pro_matrix(ERBB, NFFT=960, mode=1) # inverse ERB convert matrix
# iShane_B = Shane_B @ iERB_Matrix.T
# np.round(iShane_B,3)